# Modeling 

In [90]:
from sklearn import model_selection, preprocessing, naive_bayes, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix
from sklearn import decomposition, ensemble
import pandas as pd
import numpy as np

## Setting dataframes
- df_baseline is just numerical data
- just metadata dataframe (in the works)
- df_text is with vectorized description
    - this was already split into train and test sets in preprocessing
    - joined to df after splitting into train/test

In [3]:
# load in dataframe
df = pd.read_csv('csv_files/9_1_all_books_df.csv')

In [4]:
# this just includes pages and ratings along with the target of best seller or not
df_baseline = df[['rating', 'pages', 'best_seller']]

In [46]:
# this data is already split into the training set with a reset index
# so it will be concated with the df after spliting into train and test sets
df_text_X = pd.read_csv('csv_files/X_train_nlp.csv')
df_text_y= pd.read_csv('csv_files/y_train_nlp.csv', header=None)
df_text_X_test = pd.read_csv('csv_files/X_test_nlp.csv')
df_text_y_test = pd.read_csv('csv_files/y_test_nlp.csv', header=None)

In [62]:
df_text_X.tail()
# df_text_X.shape

,aaron,ab,abandon,abandonment,abby,abc,abduct,abducted,ability,able,...,zelda,zest,zeus,zillion,zimmer,zoe,zone,zoom,zorie,zoroastrian
1093,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1094,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1095,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1096,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
df_text_y.tail()
# df_text_y.shape

,0
1093,1
1094,1
1095,1
1096,0
1097,1


In [50]:
df_text_X_test.head()
df_text_X_test.shape

(275, 5842)

In [86]:
df_text_y_test.head()
# df_text_y_test.shape
type(df_text_y_test)

# this is a series
type(df_text_y_test[0])

pandas.core.series.Series

In [82]:
type(y_test_rating)
# this is a pandas series

pandas.core.series.Series

In [6]:
df_text_y.columns

Int64Index([0], dtype='int64')

In [7]:
# df_text_y[0]


In [8]:
# look at dataframe
print(df_baseline.columns)
print(df_baseline.shape)
df_baseline.head()

Index(['rating', 'pages', 'best_seller'], dtype='object')
(1373, 3)


,rating,pages,best_seller
0,3.56,415,0
1,4.07,346,0
2,3.47,355,0
3,3.42,368,0
4,4.35,480,0


In [9]:
# split the dataset into training and validation datasets for just numerical data
X_train_rating, X_test_rating, y_train_rating, y_test_rating = model_selection.train_test_split(df_baseline[['rating', 'pages']], 
                                                                    df_baseline['best_seller'], 
                                                                    test_size = .2,
                                                                   random_state=42)

In [10]:
# this actually doesn't need to happen... the text data is already split...
# split the dataset with nlp and other features
X_train, X_test, y_train, y_test = model_selection.train_test_split(df[['rating', 'pages']], 
                                                                    df['best_seller'], 
                                                                    test_size = .2,
                                                                   random_state=42)

In [11]:
df_text = df_text_x 

In [12]:
df_text['_target'] = df_text_y[0]

In [13]:
df_text['_target'].dtype

dtype('int64')

In [14]:
df_text.head()

,aaron,ab,abandon,abandonment,abby,abc,abduct,abducted,ability,able,...,zest,zeus,zillion,zimmer,zoe,zone,zoom,zorie,zoroastrian,_target
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# deep vs shallow copies
df_text_x = df_text_x.loc[:, df_text_x.columns != '_target']

In [16]:
y_train_rating.tail()

1095    1
1130    1
1294    1
860     0
1126    1
Name: best_seller, dtype: int64

## Numerical Baseline model
- On numerical data only

In [17]:
base = DummyClassifier(random_state=42)

In [18]:
base.fit(X_train_rating, y_train_rating)

DummyClassifier(constant=None, random_state=42, strategy='stratified')

In [19]:
print('mean training accuracy' ,base.score(X_train_rating, y_train_rating))
print('mean test accuracy', base.score(X_test_rating, y_test_rating))
# mean training accuracy 0.663023679417122
# mean test accuracy 0.6654545454545454

mean training accuracy 0.663023679417122
mean test accuracy 0.6654545454545454


# Tree based Modeling

## Numerical Random Forest

In [20]:
random_forest = RandomForestClassifier()

In [21]:
random_forest.fit(X_train_rating, y_train_rating)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [22]:
random_forest.score(X_test_rating, y_test_rating)

0.7781818181818182

In [23]:
random_forest.decision_path(X_train_rating)

(<1098x4428 sparse matrix of type '<class 'numpy.int64'>'
 	with 135177 stored elements in Compressed Sparse Row format>,
 array([   0,  463,  890, 1375, 1818, 2257, 2682, 3103, 3574, 4009, 4428]))

In [24]:
random_forest.decision_path(X_test_rating)

(<275x4428 sparse matrix of type '<class 'numpy.int64'>'
 	with 33391 stored elements in Compressed Sparse Row format>,
 array([   0,  463,  890, 1375, 1818, 2257, 2682, 3103, 3574, 4009, 4428]))

In [25]:
random_forest.feature_importances_

array([0.48417116, 0.51582884])

# Text modeling

In [27]:
random_forest_text = RandomForestClassifier()

In [28]:
random_forest_text.fit(df_text_X, df_text_y[0])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [29]:
# look at what features are important...
random_forest_text.feature_importances_
len(random_forest_text.feature_importances_)

5842

In [30]:
max(random_forest_text.feature_importances_)

0.027614468447527594

In [45]:
# random_forest_text.feature_importances_.where(max(random_forest_text.feature_importances_))
np.where(random_forest_text.feature_importances_ == max(random_forest_text.feature_importances_))
# this is 'prize'... maybe a little to telling

(array([4091]),)

In [32]:
random_forest_text.feature_importances_[4091]

0.027614468447527594

In [36]:
df_text_x.head()

,aaron,ab,abandon,abandonment,abby,abc,abduct,abducted,ability,able,...,zelda,zest,zeus,zillion,zimmer,zoe,zone,zoom,zorie,zoroastrian
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
# name of column
print(df_text_x.columns[4091])
# amount of times word appears
df_text_x.iloc[:,4091].sum()
# this is a word to remove...

prize


23

In [35]:
# look at accuracy for training set
random_forest_text.score(df_text_x, df_text_y[0])

0.970856102003643

In [89]:
random_text_predictions = random_forest_text.predict(df_text_X_test)

In [91]:
confusion_matrix(df_text_y_test[0], random_text_predictions)

array([[205,   8],
       [ 45,  17]])

# Logistic Regression

In [26]:
# log  = LogisticRegression()

# things to do
- get the text test set... check
- find most important features....
- word counts...
